# Python project: File_organizer Federico Gori

## STEP #1

In [91]:
# creazione delle sottocartelle per immagini, file audio e testo, se non già presenti nella cartella "files"

import os

#reimpostazione della directory principale
returnto=os.path.split(os.getcwd())
mainpath=os.path.split(sis[0])
os.chdir(mainpath[0])

# selezione della directory "/files" e creazione delle sottocartelle("Audio","Docs","Images")
os.chdir(os.path.join(os.getcwd(),'files'))

os.makedirs(os.path.join(os.getcwd(),'Audio'),mode = 0o777, exist_ok = True)   
os.makedirs(os.path.join(os.getcwd(),'Docs'),mode = 0o777, exist_ok = True)
os.makedirs(os.path.join(os.getcwd(),'Images'),mode = 0o777, exist_ok = True)


***

In [ ]:
# scelta delle librerie per selezionare e spostare i file, creando un csv di recap
import shutil, os, csv

# selezione della directory ""/files"
os.chdir(os.getcwd())

# funzione "locate" per semplicare il processo di selezione delle sottocartelle dove spostare i file per formato
def locate(x):
    return os.path.join(os.getcwd(),x)

# struttura dati "formats" divisa per tipologia, estensione e sottocarella di destinazione dei file per tipo
formats = {'Image':[('jpg','jpeg','png'),locate('Images')],
           'Audio':['mp3',locate('Audio')],
           'Doc': [('odt','txt'),locate('Docs')]}

# creazione del file .csv di recap
recap_exists = os.path.isfile("recap.csv")
with open("recap.csv", mode="a", newline='') as csvfile:
    fieldnames = ["name","type","size(B)"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

# script per evitare la sovrascrittura dell'header nel file .csv aggiungendo nuovi file alla cartella
    if not recap_exists:
        writer.writeheader()    

# iterazione all'interno della cartella /files
    for file in sorted(os.listdir()):

# iterazione nella struttura dati "formats" per selezionare le chiavi(tipologia di file) ed i valori(estensioni dei file, destinazione finale)
        for name,ext in formats.items():
        
# esclusione delle sotto cartelle create in precedenza in os.listdir()
            if file.endswith(ext[0]):
        
# smistamento dei file nelle sotto cartelle di appartenenza
                shutil.move(locate(file),os.path.join(ext[1],file))
    
# visualizzazione dei nomi dei file, della loro tipologia e delle loro dimensioni in Byte
# i file trasferiti sono segnati contemporaneamente nel file "recap.csv" creato precedentemente 
                size = str(os.path.getsize(os.path.join(ext[1],file)))
                print(file.split('.')[0],'type:'+ name, size +'B')
                writer.writerow({"name":file.split('.')[0],"type": name,"size(B)": size})



***

## STEP #3

In [94]:
# selezione delle librerie necessarie per la selezione delle cartelle e delle immagini contenute, la loro trasformazione in array per il calcolo dei valori medi di colore e la creazione di tabelle

from PIL import Image
from tabulate import tabulate
import os, numpy as np

# reimpostazione della directory principale dallo step #1 per lo Step #3
returnto=os.path.split(os.getcwd())
mainpath=os.path.split(sis[0])
os.chdir(mainpath[0])

# selezione della directory "Images"
os.chdir(os.path.join(os.getcwd(),'files','Images'))

# da questo script si può notare che:
#1) le immagini in grayscale hanno 2 dimensioni a contrario delle RGB e RGBA che ne hanno 3
#2) le immagini RGBA si distinguono dalle RGB rispetto alla grandezza della terza dimensione dell'immagine, che aggiunge il quarto valore "ALPHA"

'''
[in] 

for x in os.listdir():
    pic=Image.open(os.path.join(os.getcwd(),x))
    np_pic = np.array(pic)
    print(x,'| dimension = ',np.ndim(np_pic),'| shape = ',np.shape(np_pic),'| coulours_mean = ',np.mean(np_pic,axis=(0,1)))
______________________________________________________       

[out] 

bw.png | dimension =  2 | shape =  (512, 512) | coulours_mean =  21.480064392089844
daffodil.jpg | dimension =  3 | shape =  (500, 335, 3) | coulours_mean =  [109.22727164  85.52306269   4.76619104]
eclipse.png | dimension =  3 | shape =  (256, 256, 4) | coulours_mean =  [109.04953003 109.52186584  39.84899902 133.59094238]
trump.jpeg | dimension =  3 | shape =  (183, 275, 3) | coulours_mean =  [97.01080974 98.99409836 90.92379533]
'''

# creazione delle lista di liste che conterrà i dati delle immagini da inserire in tabella

images_list=[]

# iterazione attraverso il ciclo 'for' all'interno della sotto cartella "Images" 

for x in os.listdir():
# la clausola if evita che la cartella di checkpoint di Jupyter venga iterata
    if x != '.ipynb_checkpoints':

# le 4 variabili riportano il riferimento all'immagine, la trasformazione dell'immagine in array, la media dei colori e le dimensioni
        image = Image.open(os.path.join(os.getcwd(),x))
        np_image = np.array(image)
        colours_mean = np.mean(np_image, axis = (0,1))
        shape = np.shape(image)

# le clausole if isolano le immagini rispetto a:
#1) la funzione 'np.ndim', rispetto alle dimensioni delle immagini (scala di grigi = 2 | RGB,RGBA = 3)
#2) la funzione 'np.shape', rispetto alla grandezza della terza dimensione dell'immagine (RGB = 3 | RGBA = 4)

# la variabile "shape" viene suddivisa in altezza e larghezza
# le liste sono inserite all'interno della lista "images_list"
        if np.ndim(np_image)==2:
            (h,w) = shape
            images_list.append([x.split('.')[0],h,w,colours_mean,0,0,0,0])
        if np.ndim(np_image)==3 and np.shape(np_image[0,1]) == (3,):
            (h,w,d)=shape
            images_list.append([x.split('.')[0],h,w,0,colours_mean[0],colours_mean[1],colours_mean[2],0])
        if np.ndim(np_image)==3 and np.shape(np_image[0,1]) == (4,):
            (h,w,d)=shape
            images_list.append([x.split('.')[0],h,w,0,colours_mean[0],colours_mean[1],colours_mean[2],colours_mean[3]])


# creazione e visualizzazione della tabella riassuntiva
headers= ["name","height","width","grayscale","R","G","B","ALPHA"]
print(tabulate(images_list, headers, tablefmt="simple_grid",floatfmt=('.2f')))


┌──────────┬──────────┬─────────┬─────────────┬────────┬────────┬───────┬─────────┐
│ name     │   height │   width │   grayscale │      R │      G │     B │   ALPHA │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ bw       │      512 │     512 │       21.48 │   0.00 │   0.00 │  0.00 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ daffodil │      500 │     335 │        0.00 │ 109.25 │  85.56 │  4.97 │    0.00 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ eclipse  │      256 │     256 │        0.00 │ 109.05 │ 109.52 │ 39.85 │  133.59 │
├──────────┼──────────┼─────────┼─────────────┼────────┼────────┼───────┼─────────┤
│ trump    │      183 │     275 │        0.00 │  97.01 │  98.99 │ 90.92 │    0.00 │
└──────────┴──────────┴─────────┴─────────────┴────────┴────────┴───────┴─────────┘
